In [1]:
import os
import csv
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict
import pymysql

In [2]:
pr_file_path="C:/apache-tomcat-7.0.34/webapps/BestDeal/"
os.chdir('C:/Program Files/MongoDB/Server/3.2/bin')
os.system(r'mongoexport --db UserReviews --collection ProductReviews --type=csv --fields Username,ProductName,ReviewRating >'+pr_file_path+'\mongodata_train.csv')

with open(pr_file_path+"/mongodata_train.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open(pr_file_path+"/mongodata_test.csv", "w",newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)
            
file_path = os.path.expanduser(pr_file_path+'/mongodata_test.csv')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating', sep=',')

#data = Dataset.load_from_file(file_path, reader=reader)

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
items = []
for uid, user_ratings in top_n.items():
    items.append([uid, [iid for (iid, _) in user_ratings]])
    print(uid, [iid for (iid, _) in user_ratings])

out = open(pr_file_path+'/output1.csv', 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


vaish2 ['IPhone11 Pro', 'Sennheiser HD Pro', 'Apple Macbook Pro']
customer1 ['Samsung Galaxy Watch Active', 'Sennheiser HD Pro', 'Samsung 7 Series']
customer2 ['IPhone11 Pro', 'Apple Macbook Pro', 'Homido V2 Virtual reality']
customer3 ['Apple Macbook Pro', 'Samsung 7 Series', 'Sennheiser HD Pro']
customer4 ['IPhone11 Pro', 'Samsung 7 Series', 'LG C9 Series']
customer5 ['HP Spectre X360', 'Echo Plus', 'Samsung 7 Series']
customer6 ['Samsung Galaxy Watch Active', 'HP Spectre X360', 'Sennheiser HD Pro']
customer7 ['Samsung Galaxy Watch Active', 'Sennheiser HD Pro', 'Homido V2 Virtual reality']
customer8 ['IPhone11 Pro', 'Samsung Galaxy Watch Active', 'Insignia 70 4K UHD TV']
customer9 ['Apple Macbook Pro', 'IPhone11 Pro', 'LG C9 Series']
customer10 ['Apple Macbook Pro', 'Insignia 70 4K UHD TV', 'HP Spectre X360']
customer11 ['Apple Macbook Pro', 'Insignia 70 4K UHD TV', 'HP Spectre X360']
customer12 ['Apple Macbook Pro', 'LG C9 Series', 'HTC VIVE Cosmos']
customer13 ['HP Spectre X360', '

In [4]:
cnx = pymysql.connect(user='root', password='root',
                              host='127.0.0.1',
                              database='BestDealdatabase')
cursor = cnx.cursor()
query = ("SELECT loginID, Product_ID, Transaction_Status, Order_Returned FROM transactions")
cursor.execute(query)

for transaction in cursor:
    for row in items:
        if row[0] in transaction and row[0] == transaction[0]:
            for pid in row[1]:
                if pid in transaction[1]:
                    if transaction[2] == 'Disputed' or transaction[3] == 'Yes':
                        row[1].remove(pid)

out = open(pr_file_path+'/output.csv', 'w',newline='')
output=csv.writer(out)

for item in items:
    print(item)
    output.writerow(item)
    
out.close()

['vaish2', ['IPhone11 Pro', 'Sennheiser HD Pro', 'Apple Macbook Pro']]
['customer1', ['Samsung Galaxy Watch Active', 'Sennheiser HD Pro', 'Samsung 7 Series']]
['customer2', ['IPhone11 Pro', 'Apple Macbook Pro', 'Homido V2 Virtual reality']]
['customer3', ['Apple Macbook Pro', 'Samsung 7 Series', 'Sennheiser HD Pro']]
['customer4', ['IPhone11 Pro', 'Samsung 7 Series', 'LG C9 Series']]
['customer5', ['HP Spectre X360', 'Echo Plus', 'Samsung 7 Series']]
['customer6', ['Samsung Galaxy Watch Active', 'HP Spectre X360', 'Sennheiser HD Pro']]
['customer7', ['Samsung Galaxy Watch Active', 'Sennheiser HD Pro', 'Homido V2 Virtual reality']]
['customer8', ['IPhone11 Pro', 'Samsung Galaxy Watch Active', 'Insignia 70 4K UHD TV']]
['customer9', ['Apple Macbook Pro', 'IPhone11 Pro', 'LG C9 Series']]
['customer10', ['Apple Macbook Pro', 'Insignia 70 4K UHD TV', 'HP Spectre X360']]
['customer11', ['Apple Macbook Pro', 'Insignia 70 4K UHD TV', 'HP Spectre X360']]
['customer12', ['Apple Macbook Pro', 'LG